In [4]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, TIMESTAMP, JSON
from datetime import datetime, timedelta

In [16]:
def connection():
    db_params = {
    'dbname': 'rl_trade',
    'user': 'bezzonov',
    'password': 'bezzonov_rl_trade',
    'host': '46.17.100.206',
    'port': '5432'}
    conn = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')
    return conn

def shares_list_for_user(conn, table_name, threshold_date):
    query = f"""
        WITH shares_today_available as
            (
            SELECT DISTINCT ticker
            FROM {table_name}
            WHERE time = '{datetime.strftime(datetime.today() - timedelta(days=2), '%Y-%m-%d')}'
            ),

        shares_rows as
            (
            SELECT ticker, count(*) as shares_rows_num
            FROM {table_name}
            WHERE time >= '{threshold_date}'
            GROUP BY ticker
            ORDER BY count(*) DESC
            ),

        shares_most_common as
            (
            SELECT ticker
            FROM shares_rows
            WHERE shares_rows_num = (
                                    select max(shares_rows_num)
                                    from shares_rows
                                    )
            )

        select t1.ticker
        from shares_today_available t1
        inner join shares_most_common t2
        on t1.ticker = t2.ticker
    """
    data = pd.read_sql_query(query, conn)
    return data

In [ ]:
shares_list_for_user(connection(), 'hour_shares_data', '2015-01-01')['ticker'].values.tolist()

['AFLT',
 'ALRS',
 'CHMF',
 'GAZP',
 'HYDR',
 'IRAO',
 'LKOH',
 'MAGN',
 'MGNT',
 'MOEX',
 'MTSS',
 'NLMK',
 'NVTK',
 'PHOR',
 'PIKK',
 'ROSN',
 'RTKM',
 'SBER',
 'SBERP',
 'SNGS',
 'SNGSP',
 'TATN']

In [5]:
def connection():
    db_params = {
    'dbname': 'rl_trade',
    'user': 'bezzonov',
    'password': 'bezzonov_rl_trade',
    'host': '46.17.100.206',
    'port': '5432'}
    conn = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')
    return conn

In [6]:
import requests
from datetime import datetime, timedelta


def fetch_index_close_data(index, date_from):

    all_data = []
    start = 0
    limit = 100  # можно увеличить до 1000, если нужно

    while True:
        params = {
            "start": start,
            "limit": limit
        }
        url = f'https://iss.moex.com/iss/history/engines/stock/markets/index/securities/{index}.json?from={date_from}'
        response = requests.get(url, params=params)
        response.raise_for_status()
        json_data = response.json()

        history = json_data.get("history")
        if not history:
            break

        columns = history.get("columns", [])
        rows = history.get("data", [])
        if not rows:
            break

        df = pd.DataFrame(rows, columns=columns)
        all_data.append(df)

        cursor = json_data.get("cursor", {})
        total = cursor.get("TOTAL", 0)

        start += limit
        if start >= total:
            break

    if all_data:
        full_df = pd.concat(all_data, ignore_index=True)
        full_df = full_df[["TRADEDATE", "CLOSE"]]
        full_df["TRADEDATE"] = pd.to_datetime(full_df["TRADEDATE"])
        full_df = full_df.sort_values("TRADEDATE").reset_index(drop=True)
        full_df = full_df.rename(columns={"TRADEDATE" : "date", "CLOSE" : f"{index.lower()}"})
        return full_df
    else:
        return pd.DataFrame()


def get_100_days():
    start_date = datetime.strptime("2017-06-01", "%Y-%m-%d")
    today = datetime.today()

    dates = []
    current_date = start_date

    while current_date <= today:
        dates.append(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=101)
    return dates

def get_one_index_info(index, dates_from):
    df_index = pd.DataFrame()
    for date in dates_from:
        df_part = fetch_index_close_data(index, date)
        df_index = pd.concat([df_index, df_part])
    df_index = df_index.drop_duplicates(subset=['date'])
    return df_index

indexes = ["IMOEX", "RTSI", "MOEXBC", "MOEXOG", "MOEXEU", "MOEXFN"]
dates_from = get_100_days()
df_index_full = pd.DataFrame(pd.date_range(start='2017-06-01', end=pd.Timestamp.today()).tolist(), columns=['date'])


for ind in indexes:
    df_index = get_one_index_info(ind, dates_from)
    df_index_full = df_index_full.merge(df_index, how='left', on='date')

df_index_full = df_index_full.fillna(method='ffill')

C:\Users\besso\AppData\Local\Temp\ipykernel_14332\312930848.py:80: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_index_full = df_index_full.fillna(method='ffill')


In [7]:
df_index_full

,date,imoex,rtsi,moexbc,moexog,moexeu,moexfn
0,2017-06-01,1869.66,1045.63,12051.59,4611.35,1785.19,6835.91
1,2017-06-02,1881.87,1046.02,12147.12,4614.26,1795.99,6974.87
2,2017-06-03,1881.87,1046.02,12147.12,4614.26,1795.99,6974.87
3,2017-06-04,1881.87,1046.02,12147.12,4614.26,1795.99,6974.87
4,2017-06-05,1877.69,1044.34,12113.86,4611.33,1800.43,6995.56
...,...,...,...,...,...,...,...
2899,2025-05-09,2850.93,1110.68,18502.48,7832.99,1546.12,10298.31
2900,2025-05-10,2850.93,1110.68,18502.48,7832.99,1546.12,10298.31
2901,2025-05-11,2850.93,1110.68,18502.48,7832.99,1546.12,10298.31
2902,2025-05-12,2930.17,1141.16,19027.46,8049.37,1577.20,10602.85


In [9]:
indexes = ["IMOEX", "RTSI", "MOEXBC", "MOEXOG", "MOEXEU", "MOEXFN"]
df_index_full_today = pd.DataFrame([datetime.today().date()], columns=['date'])
df_index_full_today['date'] = pd.to_datetime(df_index_full_today['date'] )

for ind in indexes:
    df_index_today = get_one_index_info(ind, [datetime.strftime(datetime.today().date(), format='%Y-%m-%d')])
    df_index_today['date'] = pd.to_datetime(df_index_today['date'])
    df_index_full_today = df_index_full_today.merge(df_index_today, how='left', on='date')

KeyError: 'date'

In [ ]:
df_index_full_today

,date,imoex,rtsi,moexbc,moexog,moexeu,moexfn
0,2025-05-12,2930.17,1141.16,19027.46,8049.37,1577.2,10602.85


In [11]:
df_index_full.to_sql('stock_market_indexes', con=connection(), if_exists='replace', index=True, dtype={'date': TIMESTAMP})


904